In [1]:
import chromadb
import uuid
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import numpy as np
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
import pickle

In [2]:
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3296\3002436811.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [6]:
loaders = [
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\BOCYL-D-17052024-2 2024.pdf"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\decreto de infantil 30 de octubre 29 DE SEPTIEMBRE.pdf"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\Decreto primaria octubre 29 SEP 2022.pdf"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\Decreto secundaria octubre.pdf"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\Franz_Kafka_The_Metamorphosis.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size= 2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size= 400)

coleccion = "prueba_kafka_child"


In [8]:
# from langchain_core.documents import Document

# chunks = [Document(page_content="Hola", metadata={"page": 3333})]

In [4]:
client = chromadb.HttpClient("localhost", 7888)

collection = client.get_or_create_collection(coleccion)

display(client.list_collections())
collection = Chroma(collection_name=coleccion, client=client,  embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"), persist_directory=r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\desarrollo\prueba")

store = InMemoryStore() 

[Collection(name=prueba_kafka),
 Collection(name=prueba_kafka_large),
 Collection(name=prueba_kafka_child)]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3296\678315488.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  collection = Chroma(collection_name=coleccion, client=client,  embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"), persist_directory=r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\desarrollo\prueba")


In [6]:
retriever = ParentDocumentRetriever(
    vectorstore=collection,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [11]:
retriever.add_documents(docs)

In [28]:
with open("parent_store.pkl", 'wb') as outp:
    pickle.dump(store, outp, pickle.HIGHEST_PROTOCOL)

In [5]:
with open("parent_store.pkl", 'rb') as inp:
    store = pickle.load(inp)

In [7]:
len(list(store.yield_keys()))

2771

In [8]:
np.unique(np.array([ x["source"] for x in collection.get()["metadatas"]])), len(collection.get()["ids"]), collection.get()["data"]

(array(['C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto primaria octubre 29 SEP 2022.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto secundaria octubre.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Franz_Kafka_The_Metamorphosis.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\decreto de infantil 30 de octubre 29 DE SEPTIEMBRE.pdf'],
       dtype='<U115'),
 19701,
 None)

In [31]:
#client.delete_collection("prueba_kafka")

In [9]:
collection.similarity_search("Es aceptable tener una única clase con 35 alumnos de primaria?")

[Document(metadata={'doc_id': 'b72dc07a-32b4-4dc1-9e25-4f9f631a3b4e', 'page': 5, 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf'}, page_content='1. En virtud de lo dispuesto en el artículo 19.9 del Decreto 38/2022, de 29 de \nseptiembre, cuando el progreso del alumnado no sea el adecuado se establecerán \nmedidas de refuerzo educativo.\nEstas medidas deberán adoptarse en cualquier momento del curso, tan pronto \ncomo se detecten las dificultades y estarán dirigidas a garantizar la adquisición de los'),
 Document(metadata={'doc_id': 'b7cca998-902a-4828-8248-961f52bc7fd8', 'page': 35, 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto primaria octubre 29 SEP 2022.pdf'}, page_content='Boletín Oficial de Castilla y León\nNúm. 190 Pág. 48351Viernes, 30 de septiembre de 2022\nCompetencia personal, social y de aprender a aprender (CPSAA) \nDescriptores operativos: \n \n  \nAl completar la educaci

In [10]:
retriever.invoke("Es aceptable tener una única clase con 35 alumnos de primaria?")

[Document(metadata={'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf', 'page': 5}, page_content='Boletín Oficial de Castilla y León\nNúm. 95 Pág. 22Viernes, 17 de mayo de 2024\nArtículo 9. Medidas de refuerzo educativo, plan específico de refuerzo, plan de \nrecuperación y plan de enriquecimiento curricular.\n1. En virtud de lo dispuesto en el artículo 19.9 del Decreto 38/2022, de 29 de \nseptiembre, cuando el progreso del alumnado no sea el adecuado se establecerán \nmedidas de refuerzo educativo.\nEstas medidas deberán adoptarse en cualquier momento del curso, tan pronto \ncomo se detecten las dificultades y estarán dirigidas a garantizar la adquisición de los \naprendizajes imprescindibles para continuar el proceso educativo.\nSe planificarán por cualquiera de los docentes que atienden al alumno teniendo \nen cuenta las necesidades que se detecten a partir de la información recogida en todo \nmomento acerca del proceso de en